In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import matplotlib.pyplot as plt

In [2]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')

In [3]:
base_model.trainable = True

In [4]:
def define_model():
    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(4, activation='softmax'))
    return model

if __name__ == '__main__':
    model = define_model()
    print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 4)                 5124      
Total params: 2,263,108
Trainable params: 2,228,996
Non-trainable params: 34,112
_________________________________________________________________
None


In [5]:
sgd = tf.keras.optimizers.SGD(lr=1e-6, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy' , metrics=['accuracy'])

In [6]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [7]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

training_set = train_datagen.flow_from_directory('D3/train/', 
                                                 target_size = (224,224), 
                                                 batch_size = 10, 
                                                 class_mode = 'categorical', 
                                                 shuffle = True)

validation_set = test_datagen.flow_from_directory('D3/validation/', 
                                            target_size = (224,224), 
                                            batch_size = 1, 
                                            class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('D3/test/', 
                                            target_size = (224,224), 
                                            batch_size = 1, 
                                            class_mode = 'categorical')

Using TensorFlow backend.


Found 5694 images belonging to 4 classes.
Found 1140 images belonging to 4 classes.
Found 760 images belonging to 4 classes.


In [8]:
checkpoint_path = "training_4/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [9]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience = 3)

In [10]:
training = model.fit(training_set, 
                     epochs = 30, 
                     validation_data = validation_set, 
                     verbose = 1, 
                     callbacks = [cp_callback, early_stopping])

Train for 570 steps, validate for 1140 steps
Epoch 1/30
569/570 [============================>.] - ETA: 1s - loss: 1.3724 - accuracy: 0.3482
Epoch 00001: saving model to training_007/cp.ckpt
570/570 [==============================] - 793s 1s/step - loss: 1.3723 - accuracy: 0.3488 - val_loss: 1.3492 - val_accuracy: 0.3904
Epoch 2/30
569/570 [============================>.] - ETA: 1s - loss: 1.2042 - accuracy: 0.4731
Epoch 00002: saving model to training_007/cp.ckpt
570/570 [==============================] - 816s 1s/step - loss: 1.2041 - accuracy: 0.4731 - val_loss: 1.0697 - val_accuracy: 0.5588
Epoch 3/30
569/570 [============================>.] - ETA: 1s - loss: 1.0697 - accuracy: 0.5806
Epoch 00003: saving model to training_007/cp.ckpt
570/570 [==============================] - 814s 1s/step - loss: 1.0693 - accuracy: 0.5810 - val_loss: 0.9264 - val_accuracy: 0.6763
Epoch 4/30
569/570 [============================>.] - ETA: 0s - loss: 0.9587 - accuracy: 0.6606
Epoch 00004: saving model

569/570 [============================>.] - ETA: 0s - loss: 0.2451 - accuracy: 0.9592
Epoch 00030: saving model to training_007/cp.ckpt
570/570 [==============================] - 504s 884ms/step - loss: 0.2449 - accuracy: 0.9593 - val_loss: 0.1446 - val_accuracy: 0.9877


In [11]:
results = model.evaluate(test_set)

760/760 [==============================] - 84s 111ms/step - loss: 0.1521 - accuracy: 0.9908


In [12]:
model.save('saved_model/MobileNet')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model/MobileNet007/assets


In [13]:
converter = tf.lite.TFLiteConverter.from_saved_model('saved_model/MobileNet')
tflite_model = converter.convert()
open("converted_model_mobilenet.tflite", "wb").write(tflite_model)

8881136